
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [7]:

# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.

# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.

# Part 1

# Load the data Make sure that all your data is numpy arrays and not pandas dataframes or series.
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score
import numpy as np
import math

# Load the dataset
# data_path = '/mnt/data/Corona_NLP_train.csv'
df = pd.read_csv("data/Corona_NLP_train.csv", encoding='latin')
# df = pd.read_csv("C:\\Users\\Perpetua\\Downloads\\Corona_NLP_train.csv", encoding='latin')

# Filter out Neutral sentiment
df_filtered = df[df['Sentiment'] != 'Neutral']

# Map sentiment to binary values: 1 for Positive, 0 for Negative
df_filtered['Y'] = df_filtered['Sentiment'].apply(lambda x: 1 if 'Positive' in x else 0)

# Define X (features) and Y (target)
#X = df_filtered['OriginalTweet'].to_numpy()
#Y = df_filtered['Y'].to_numpy()


X = df_filtered['OriginalTweet'].to_numpy()
Y = df_filtered['Y'].to_numpy()



# Split the data into train (60%), temp (40% for test+validation)
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, train_size=0.6, random_state=42, stratify=Y)

# Split the remaining data into test (15%) and validation (25%)
test_size = 0.15 / (0.15 + 0.25)  # Adjusted proportion for test split
X_test, X_val, Y_test, Y_val = train_test_split(X_temp, Y_temp, test_size=test_size, random_state=42, stratify=Y_temp)

# Convert all data splits to numpy arrays (ensure consistency)
# X_train = np.array(X_train)
# X_test = np.array(X_test)
# X_val = np.array(X_val)
# Y_train = np.array(Y_train)
# Y_test = np.array(Y_test)
# Y_val = np.array(Y_val)


X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)
X_valid = np.array(X_val)
Y_valid = np.array(Y_val)


df_filtered.head()

C:\Users\Perpetua\AppData\Local\Temp\ipykernel_8784\933513166.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Y'] = df_filtered['Sentiment'].apply(lambda x: 1 if 'Positive' in x else 0)


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,Y
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive,1
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive,1
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive,1
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative,0
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,Positive,1


In [8]:

# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.



# Define and train the model pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

model = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Train the model
model.fit(X_train, Y_train)

# Output the shapes of the splits
print("Train set size:", X_train.shape)
print("Test set size:", X_test.shape)
print("Validation set size:", X_val.shape)

# Verify class distribution in each split
print("\nTrain class distribution:\n", np.unique(Y_train, return_counts=True))
print("Test class distribution:\n", np.unique(Y_test, return_counts=True))
print("Validation class distribution:\n", np.unique(Y_val, return_counts=True))


Train set size: (20066,)
Test set size: (8361,)
Validation set size: (5017,)

Train class distribution:
 (array([0, 1], dtype=int64), array([ 9239, 10827], dtype=int64))
Test class distribution:
 (array([0, 1], dtype=int64), array([3849, 4512], dtype=int64))
Validation class distribution:
 (array([0, 1], dtype=int64), array([2310, 2707], dtype=int64))


In [9]:

# Part 3

# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.


# Evaluate precision and recall on the test set
Y_pred = model.predict(X_test)

precision_0 = precision_score(Y_test, Y_pred, pos_label=0)
precision_1 = precision_score(Y_test, Y_pred, pos_label=1)
recall_0 = recall_score(Y_test, Y_pred, pos_label=0)
recall_1 = recall_score(Y_test, Y_pred, pos_label=1)

# Print precision and recall values
print("Precision for class 0:", precision_0)
print("Precision for class 1:", precision_1)
print("Recall for class 0:", recall_0)
print("Recall for class 1:", recall_1)




Precision for class 0: 0.8697389451251998
Precision for class 1: 0.8732363794226178
Recall for class 0: 0.8482722785138997
Recall for class 1: 0.8916223404255319


In [11]:

# Part 4

def cost(model,threshold,X,Y):
    # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost
    
     # Predict probabilities
    probabilities = model.predict_proba(X)[:, 1]  # Probabilities for class 1
    
    # Make predictions based on threshold
    predictions = (probabilities >= threshold).astype(int)
    
    # Calculate costs
    cost = 0
    for y_true, y_pred in zip(Y, predictions):
        if y_true == 1 and y_pred == 0:  # False Negative
            cost += 1
        elif y_true == 0 and y_pred == 1:  # False Positive
            cost += 5
    
    # Return average cost
    
    return cost / len(Y)

# Example usage of the cost function
threshold = 0.5
test_cost = cost(model, threshold, X_test, Y_test)
print("Average cost on test set:", test_cost)


Average cost on test set: 0.40772634852290396


In [12]:

# Part 5

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.


# Find the optimal threshold
thresholds = np.linspace(0, 1, 101)  # Thresholds from 0 to 1 in steps of 0.01
costs = [cost(model, t, X_test, Y_test) for t in thresholds]
optimal_threshold = thresholds[np.argmin(costs)]
cost_at_optimal_threshold = min(costs)

print("Optimal threshold:", optimal_threshold)
print("Cost at optimal threshold:", cost_at_optimal_threshold)

# optimal_threshold = XXX
# cost_at_optimal_threshold = XXX

Optimal threshold: 0.84
Cost at optimal threshold: 0.2796316230116015


In [13]:

# Part 6


# Compute cost on the validation set
cost_at_optimal_threshold_valid = cost(model, optimal_threshold, X_val, Y_val)

# Compute confidence interval using Hoeffding's inequality
n = len(Y_val)
confidence = 0.99
epsilon = math.sqrt(math.log(2 / (1 - confidence)) / (2 * n))
cost_interval_valid = (cost_at_optimal_threshold_valid - epsilon, cost_at_optimal_threshold_valid + epsilon)

print("Cost on validation set at optimal threshold:", cost_at_optimal_threshold_valid)
print("99% Confidence interval for cost:", cost_interval_valid)




# cost_at_optimal_threshold_valid = XXX
# cost_interval_valid = XXX

assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

Cost on validation set at optimal threshold: 0.26310544149890375
99% Confidence interval for cost: (0.24012639859344073, 0.28608448440436673)


In [14]:

# Part 7


# Compute empirical variance and Bennett's inequality confidence interval
def empirical_variance(costs, mean_cost):
    return np.mean((costs - mean_cost) ** 2)

# Calculate the cost values for each validation example
probabilities_val = model.predict_proba(X_val)[:, 1]
predictions_val = (probabilities_val >= optimal_threshold).astype(int)
costs_val = [(1 - pred) * y + 5 * (1 - y) * pred for y, pred in zip(Y_val, predictions_val)]
mean_cost_val = np.mean(costs_val)
variance_of_C = empirical_variance(np.array(costs_val), mean_cost_val)

# Bennett's inequality for confidence interval
b = 5  # Upper bound for individual cost
delta = 0.01  # Confidence level (1 - 0.99)
confidence_bound = (b / n) * math.log(2 / delta)
bennett_interval = (mean_cost_val - confidence_bound, mean_cost_val + confidence_bound)

interval_of_C = bennett_interval

print("Empirical variance of C:", variance_of_C)
print("Confidence interval of C using Bennett's inequality:", interval_of_C)




# variance_of_C = XXX
# interval_of_C = XXX


assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

Empirical variance of C: 0.5446882234844423
Confidence interval of C using Bennett's inequality: (0.2578250773703927, 0.2683858056274148)
